# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [2]:
pl.read_ndjson('../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3...","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3...","""Unknown""",669,"""Laurianneshire...","""AutoLoans""","""Accepted"""
"""MoneyMarketSav...","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh...","""DepositAccount...","""Accepted"""
"""UPlusFinPerson...","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount...","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [3]:
anon = DataAnonymization(hds_folder='../../data')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [4]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,CK_PREDICTOR_0,IH_PREDICTOR_0,OUTCOME
f64,str,str,str,str,bool
0.983033,"""77659182060560...","""17687376713516...","""85472606184047...","""37025886780024...",false
0.144221,"""35727389419330...","""14831975152123...","""85472606184047...","""18061424597309...",true
0.679745,"""10160713527128...","""43548773850200...","""12127579634181...","""37025886780024...",false
1.0,"""35727389419330...","""15527205212760...","""12210928210416...","""37025886780024...",false
0.718982,"""77659182060560...","""19376293798010...","""12210928210416...","""46932658744559...",true
0.0,"""10160713527128...","""16522686778959...","""70180634670251...",null,false
0.02333,"""10160713527128...","""56542100586670...","""12210928210416...","""46932658744559...",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [5]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Context_Name': 'CK_PREDICTOR_0',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'OUTCOME'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [6]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': True,
 'mask_ih_names': True,
 'mask_outcome_name': True,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime', 'Decision_OutcomeTime']}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [7]:
anon = DataAnonymization(hds_folder='../../data', mask_ih_names=False, mask_outcome_values=False, mask_context_key_values=False, mask_context_key_names=False)
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,OUTCOME
f64,str,str,str,str,str
0.983033,"""26147176613051...","""13749488081303...","""FirstMortgage3...","""14921383665438...","""Rejected"""
0.144221,"""71226778007428...","""88612884249396...","""FirstMortgage3...","""98996640880559...","""Accepted"""
0.679745,"""10609058199223...","""15135653830369...","""MoneyMarketSav...","""14921383665438...","""Rejected"""
1.0,"""71226778007428...","""86084059998400...","""BasicChecking""","""14921383665438...","""Rejected"""
0.718982,"""26147176613051...","""94223062024558...","""BasicChecking""","""84282893704966...","""Accepted"""
0.0,"""10609058199223...","""16594334543608...","""UPlusFinPerson...",null,"""Rejected"""
0.02333,"""10609058199223...","""12843537306871...","""BasicChecking""","""84282893704966...","""Rejected"""


The configs can also be written and read as such:

In [8]:
anon.config.save_to_config_file('config.json')

In [9]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,OUTCOME
f64,str,str,str,str,str
0.983033,"""11893116425855...","""11828761312301...","""FirstMortgage3...","""78977854655358...","""Rejected"""
0.144221,"""26649976646227...","""73047212535520...","""FirstMortgage3...","""11290340834939...","""Accepted"""
0.679745,"""10514599028942...","""13016209124633...","""MoneyMarketSav...","""78977854655358...","""Rejected"""
1.0,"""26649976646227...","""37494587438103...","""BasicChecking""","""78977854655358...","""Rejected"""
0.718982,"""11893116425855...","""30566918725785...","""BasicChecking""","""34242959334950...","""Accepted"""
0.0,"""10514599028942...","""10779814688149...","""UPlusFinPerson...",null,"""Rejected"""
0.02333,"""10514599028942...","""16474810134164...","""BasicChecking""","""34242959334950...","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [10]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_City=PREDICTOR_2
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=OUTCOME



In [11]:
anon.write_to_output(ext='arrow')

In [12]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,OUTCOME
f64,str,str,str,str,str
0.983033,"""13901847100685...","""20790230098784...","""FirstMortgage3...","""18080585646216...","""Rejected"""
0.144221,"""23684443556836...","""11260486451715...","""FirstMortgage3...","""57688128632934...","""Accepted"""
0.679745,"""64145511274717...","""70713395177550...","""MoneyMarketSav...","""18080585646216...","""Rejected"""
1.0,"""23684443556836...","""15333562304630...","""BasicChecking""","""18080585646216...","""Rejected"""
0.718982,"""13901847100685...","""83744299430110...","""BasicChecking""","""11044393919249...","""Accepted"""
0.0,"""64145511274717...","""10282151294859...","""UPlusFinPerson...",null,"""Rejected"""
0.02333,"""64145511274717...","""10797654301730...","""BasicChecking""","""11044393919249...","""Rejected"""
